In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/PROGETTO-PIRELLI/datasets/resnet_dataset.zip &> /dev/null

# Importing Libraries

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from PIL import Image
from transformers import ViTFeatureExtractor, ViTForImageClassification
from sklearn.metrics import f1_score
import numpy as np
from tqdm import tqdm

# Dataset

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.img_folder = os.path.join(root_dir, "imgs")
        self.label_folder = os.path.join(root_dir, "labels")
        self.image_files = [f for f in os.listdir(self.img_folder) if os.path.isfile(os.path.join(self.img_folder, f))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_folder, img_name)
        label_path = os.path.join(self.label_folder, img_name.replace(".png", ".txt"))

        # Load image
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        # Load label from text file
        with open(label_path, 'r') as file:
            label_str = file.read().strip()
            if label_str == "0":
              label = torch.tensor([1.,0.])
            elif label_str == "1":
              label = torch.tensor([0.,1.])
            else:
              label = torch.tensor([1.,1.])

        return image, label

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define data transformations
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = CustomDataset(root_dir="/content/new_dataset/train", transform=data_transform)
val_dataset = CustomDataset(root_dir="/content/new_dataset/train", transform=data_transform)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


num_classes = len(train_dataset[0][1])
print("Num classes ",num_classes)


Num classes  2


# Model Definition and Training

In [ ]:
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224').to(device)
model.classifier = nn.Linear(model.config.hidden_size, num_classes).to(device)
# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Adjust the loss function for binary classification
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

## Training

In [ ]:
def evaluate(model, val_loader):
    model.eval()
    val_predictions = []
    val_targets = []

    with torch.no_grad():
        for val_inputs, val_labels in tqdm(val_loader, desc='Evaluation'):
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)

            val_outputs = model(val_inputs)

            # Store predictions and targets for later evaluation
            val_predictions.extend(val_outputs.logits.cpu().numpy())
            val_targets.extend(val_labels.cpu().numpy())

    val_predictions = np.array(val_predictions)
    # Convert predictions and targets to numpy arrays
    val_predictions = torch.sigmoid(torch.Tensor(val_predictions)).numpy()
    val_targets = np.array(val_targets)

    # Calculate F1-score on the validation set
    f1 = f1_score(val_targets, (val_predictions > 0.5).astype(int), average='macro')
    return f1

In [ ]:
# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, labels in tqdm(train_loader, desc='Training'):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_train_loss = total_loss / len(train_loader)
    print("")

    f1 = evaluate(model, val_loader)


    print(f"\nEpoch {epoch + 1}/{num_epochs}, Train Loss: {average_train_loss:.3f}, Validation F1-score: {f1:.3f}\n")




Training: 100%|██████████| 22/22 [00:32<00:00,  1.47s/it]


Evaluation: 100%|██████████| 22/22 [00:15<00:00,  1.40it/s]



Epoch 1/3, Train Loss: 0.328, Validation F1-score: 0.967



Training: 100%|██████████| 22/22 [00:31<00:00,  1.44s/it]


Evaluation: 100%|██████████| 22/22 [00:15<00:00,  1.44it/s]



Epoch 2/3, Train Loss: 0.101, Validation F1-score: 0.993



Training: 100%|██████████| 22/22 [00:32<00:00,  1.46s/it]


Evaluation: 100%|██████████| 22/22 [00:15<00:00,  1.43it/s]


Epoch 3/3, Train Loss: 0.045, Validation F1-score: 0.997



## Test set evaluation

In [ ]:
test_dataset = CustomDataset(root_dir="/content/new_dataset/test", transform=data_transform)

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
f1 = evaluate(model,test_loader)
print(f"\n\nTest F1-score: {f1:.3f}")

Evaluation: 100%|██████████| 9/9 [00:06<00:00,  1.33it/s]



Test F1-score: 0.975


## Extracting The model without classification head and saving

In [ ]:
class ModelWithoutHead(nn.Module):
    def __init__(self, model):
        super(ModelWithoutHead, self).__init__()
        self.features = model

    def forward(self, x):
        return self.features(x).last_hidden_state

In [ ]:
model_without_head = nn.Sequential(*list(model.children())[:-1])

In [ ]:
torch.save(model_without_head.state_dict(), '/content/drive/MyDrive/PROGETTO-PIRELLI/visual.pth')

### Loading and Testing For CAUSE

In [ ]:
# Load pre-trained ViT model
model_test = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
model_test.classifier = nn.Linear(model.config.hidden_size, num_classes)
model_without_head_test = nn.Sequential(*list(model_test.children())[:-1])

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/PROGETTO-PIRELLI/visual.pth')

# Load the state dictionary into your model
model_without_head_test.load_state_dict(checkpoint)

<All keys matched successfully>

In [ ]:
model_cause  = ModelWithoutHead(model_without_head_test)

In [ ]:
model_cause.to(device)
for test_input, test_labels in test_loader:
  test_input, test_labels = test_input.to(device), test_labels.to(device)
  output = model_cause(test_input)
  print(type(output))
  print(output.shape)
  break

<class 'torch.Tensor'>
torch.Size([32, 197, 768])
